Lors de ce TP, nous allons explorer la notion d'information mutuelle dans le cadre d'une tâche de classification de mails :  un mail est-il un spam ou un ham  ? 

Pour manipuler les arbres de décision et les données, nous allons utiliser sklearn (ou scikit-learn) dont la page web est : http://scikit-learn.org/stable/






# Les données

Récupérer les données (lingspam.p.zip) et décompresser l'archive. L'archive est un pickle des données déjà préparées. 
Voici du code pour lire et mettre en forme les donnéés (avec les imports aussi): 

In [1]:
import pickle, os 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz

train_texts, test_texts, train_labels, test_labels = pickle.load(open("lingspam.p", "rb" ), encoding='latin1')


- De quels types sont les variables  train_texts, test_texts, train_labels, test_labels ? 
- leurs dimensions ? 
- De quels types sont les données stockées ? 
- Combien y-a-t-il d'exemple pour l'apprentissage et le test  ?

## Matrice terme/document 

Nous allons maintenant transformer les données en utilisant sklearn. Pour cela il existe un outil: CountVectorizer
Cet objet permet de générer une matrice de compte. Nous allons utiliser la version binaire: chaque caractéristique est binaire, indépendante des autres caractéristique et représente la présence (ou l'absence) d'un mot dans un mail.  Pour cela nous procédons en 2 étapes: 


In [64]:
'''
les type des variables train_texts, test_texts, train_labels, test_labels sont <class 'numpy.ndarray'>

leurs dimensions sont (2604,) (2604,) (290,) (290,)

type de train_texts et train_labels est contents de e-mail

type de train_labels et test_labels est valeur binaire

'''

print(type(train_texts), type(train_labels))
print(train_texts.shape, train_labels.shape, test_texts.shape, test_labels.shape)
print(train_texts[0])
print(train_labels[:5])

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(2604,) (2604,) (290,) (290,)
b'Subject: pragmatics / extended deadline\n\nextended deadline - december 1 , 1998 : call for papers pragma99 international pragmatics conference on pragmatics and negotiation june 13-16 , 1999 tel aviv university and hebrew university of jerusalem tel aviv and jerusalem israel the main theme of this conference is the pragmatics of negotiation , interpreted in a very broad sense . interlocutors engage in negotiations about every aspect of their interaction - such as floor access and topic selection , contextual assumptions , conversational goals , and the ( mis ) interpretation and repair of their messages . topics such as cross-cultural and cross-gender ( mis ) communications , conversational procedures in disputes and collaborations , argumentation practices , and effects of assumptions and goals on the negotiating strategies of interlocutors are of special interest for this conference . the conference will

In [11]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
x_train = cv.fit_transform(train_texts) # étape 1 
x_test = cv.transform(test_texts) # étape 2 


Regarder la documentation de sklearn sur CountVectorizer et répondre aux questions suivantes

## Questions sur les données (refresher)

- Comment expliquer ces deux étapes ? 
- Combien y-a-t-il de document dans les données de train ? 
- Combien y-a-t-il de document dans les données de test ? 
- Quel est le vocabulaire ? 
- Afficher le vecteur associé au premier document d'apprentissage? 
- Quel est le type de l'objet x_train ? Expliquer ce choix ? 
- Comment trouver le lien entre un mot et son indice ? 

In [68]:
'''
1 etape: learn the vocabulary dictionary and return ter-document matrix
2 etape: extract token counts out of raw text documents using the vocabulary fitted with fit_transform(or the one provided to the constructor)

57752 de document dans les donne de train et test

cv.get_feature_names()

0 et 1, la parametre 'binary' est met par True, donc all non zero counts are set to 1

l'objet x_train est scipy.sparse.csr.csr_matrix

x_train.toarray() ou 1 intique que la ligne a le mot qui est dans la meme indice de feature_names

'''
type(x_train)

scipy.sparse.csr.csr_matrix

# Arbre de décision

La classe pour apprendre un arbre de décision est DecisionTreeClassifier. Regarder la documentation de cette classe et commenter la ligne suivante: 

In [29]:
'''
criterion: the function to measure the quality of a split, "gini" or "entropy" here is "entropy" for the information gain

splitter: the strategy used to choose the split at each node, random or best 

random_state: the seed used by the random number generator, this has to be fixed to obtain a deterministic behaviour during fitting
'''

classifier = DecisionTreeClassifier(criterion="entropy", splitter='best', random_state = 0)

- apprendre l'arbre de décision sur les données d'apprentissage
- comment connaitre la prédiction du modèle pour le premier document de test ? 
- la réponse du modèle est-elle correcte ? 
- Calculer le score de bonne prédiction sur les données de test. 
- Afficher les 10  caractéristiques sélectionnnés comme les plus importantes lors de la contruction de l'arbre de décision. 
- Regarder l'importance pour toutes les caractéristiques, que constatez-vous ? 


In [30]:
# apprendre l'arbre de décision sur les données d'apprentissage:
classifier.fit(x_train, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [46]:
import sys
import numpy as np
# for show the tree constructed by DecisionTreeClassifier
from collections import Counter
def showTree(dt, dictionary):
    left   = dt.tree_.children_left
    right  = dt.tree_.children_right
    thresh = dt.tree_.threshold
    feats  = [ dictionary[i] for i in dt.tree_.feature ]
    value  = dt.tree_.value
    def showTree_(node, s, depth):
        for i in range(depth-1):
            sys.stdout.write('|    ')
        if depth > 0:
            sys.stdout.write('-')
            sys.stdout.write(s)
            sys.stdout.write('-> ')
        if thresh[node] == -2: # leaf
            print('class {}\t({} for class 0, {} for class 1)'.format(np.argmax(value[node]), value[node][0,0], value[node][0,1]))
        else: # internal node
            print('{}?'.format(feats[node]))
            showTree_(left[ node], 'N', depth+1)
            showTree_(right[node], 'Y', depth+1)

    showTree_(0, '', 0)

In [47]:
showTree(classifier,cv.get_feature_names() )

language?
-N-> free?
|    -N-> you?
|    |    -N-> share?
|    |    |    -N-> click?
|    |    |    |    -N-> visit?
|    |    |    |    |    -N-> exclusive?
|    |    |    |    |    |    -N-> interviews?
|    |    |    |    |    |    |    -N-> xxx?
|    |    |    |    |    |    |    |    -N-> equitybuilder?
|    |    |    |    |    |    |    |    |    -N-> gino?
|    |    |    |    |    |    |    |    |    |    -N-> class 1	(0.0 for class 0, 536.0 for class 1)
|    |    |    |    |    |    |    |    |    |    -Y-> class 0	(1.0 for class 0, 0.0 for class 1)
|    |    |    |    |    |    |    |    |    -Y-> class 0	(1.0 for class 0, 0.0 for class 1)
|    |    |    |    |    |    |    |    -Y-> class 0	(1.0 for class 0, 0.0 for class 1)
|    |    |    |    |    |    |    -Y-> but?
|    |    |    |    |    |    |    |    -N-> class 0	(2.0 for class 0, 0.0 for class 1)
|    |    |    |    |    |    |    |    -Y-> class 1	(0.0 for class 0, 1.0 for class 1)
|    |    |    |    |    |    -Y->

In [91]:
importrance = classifier.feature_importances_
features = cv.get_feature_names()
# show the first num important features 
def showImportantFeature(importrance, features, num = 10):
    fea_import = []
    for i in range(len(importrance)):
        fea_import.append((features[i], importrance[i]))
    fea_import = sorted(fea_import, key=lambda x:x[1], reverse=True)
    for i in range(num):
        print('%s : %f'%(fea_import[i][0], fea_import[i][1]))
    return fea_import
_ = showImportantFeature(importrance, features)

language : 0.263709
free : 0.177927
you : 0.077782
university : 0.070938
linguistics : 0.050770
edu : 0.029252
english : 0.027605
your : 0.024322
remove : 0.023290
click : 0.020828


In [31]:
classifier.score(x_test,test_labels)

0.9586206896551724

## Profondeur de l'arbre 

Effectuer un apprentissage, puis évaluer le modèle avec différentes profondeurs maximum d'arbre (1,2,5,10,15,20,100,None): 
- construire le classifieur
- calculer son score sur le test
Une profondeur maximum de *None*  signifie qu'il n'y a pas de limitation. 
- Que constatez vous ? 

In [83]:
def profondeur(train_set, train_label, test_set, test_label,l):
    for i in l:
        c = DecisionTreeClassifier(criterion="entropy", splitter='best', random_state = 0, max_depth=i)
        c.fit(train_set, train_label)
        print('score in test set : %f with the profondeur : '%(c.score(test_set, test_label)), i)
profondeur(x_train, train_labels, x_test, test_labels, [1,2,5,10,15,20,100,None])

score in test set : 0.827586 with the profondeur :  1
score in test set : 0.917241 with the profondeur :  2
score in test set : 0.931034 with the profondeur :  5
score in test set : 0.962069 with the profondeur :  10
score in test set : 0.958621 with the profondeur :  15
score in test set : 0.958621 with the profondeur :  20
score in test set : 0.958621 with the profondeur :  100
score in test set : 0.958621 with the profondeur :  None


## Exploration du modèle
Partons du modèle dont la profondeur n'est pas fixée.
- Quelle est la profondeur de l'arbre construit ? 
- Combien avait-il de features dans les données ? 
- Combien de features sont considérées par le modèle ? 
- Quels sont les seuils de décision appliqués pour ces features ? Commenter les résultats obtenus. 


In [87]:
print(classifier.max_features)


None


## Visualisation de l'arbre
Il est possible de générer une figure qui représente l'arbre ainsi: 

In [57]:
from sklearn.tree import  export_graphviz
export_graphviz(classifier, out_file = "unNomDeFichier", feature_names = cv.get_feature_names())


Puis vous pouvez convertir le fichier en pdf grâce à la commande unix suivante : 
 dot -Tpdf unNomDeFichier  -o maFigure.pdf

- Visualiser l'arbre 
- Regarder les seuils de décision  appliqués ? Expliquer les valeurs obtenues. 

# Information mutuelle 

A partir des données d'apprentissage, calculer: 
- l'entropie de la variable Y (qui représente la classe)
- l'information mutuelle entre Y et chaque caractéristique binaire
- quels sont les 10 caractéristiques (mots) ayant l'information mutuelle la plus élevée. 

Remarque: Pour accéder facilement aux comptes, il suffit d'apprendre un Bayésien Naif de type Bernoulli avec sklearn. L'objet résultant contient les comptes. Recouper ces résultats avec ceux obtenus avec l'arbre de décision. 


In [88]:
proba_1 = sum(train_labels)/len(train_labels)
proba_0 = 1 - proba_1

entropy_y = 0 - (proba_0*np.log(proba_0)+proba_1*np.log(proba_1))
print(entropy_y)

0.6349050956769284
